# 2024-2 URP assignment

In [16]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
import re
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import time

In [17]:
# 데이터프레임 선언
Q_df = pd.DataFrame(columns = ['문서 번호', '제목',  '질문',  "위치 고유번호" , '조회수', '질문 날짜'])
A_df = pd.DataFrame(columns = ['문서 번호', '답변 ID',  '답변', '답변 날짜'])
loc_dict = pd.read_csv("naver_kin_regions_df_l2.csv", index_col = "지역 코드")

# URL에서 dirID 값을 가져오는 함수

In [18]:
def get_dirid(url):
    match = re.search(r'dirId=12(\d+)', url)
    dirid_value = match.group(1)
    return dirid_value

# URL에서 docID 값을 가져오는 함수

In [19]:
def get_docid(url):
    match = re.search(r'docId=(\d+)', url)
    dirid_value = match.group(1)
    return dirid_value

# 반복되는 패턴을 통해서 docID를 통해 위치 반환하는 함수(추후 매핑 테이블을 이용할 예정)

In [20]:
# dirID -> loc
def get_loc(id=""):
    url = 'https://kin.naver.com/qna/list.naver?dirId=12'+id

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, 'html.parser')

    location = soup.find(attrs={'class': 'location'}).get_text(strip=True)
    return location

In [21]:
def find_pattern(text):
    # 문자열 길이의 절반까지 탐색(더 긴 패턴은 두 번 반복될 수 없으므로)
    for length in range(1, len(text)):
        # 길이 `length`만큼 패턴을 잘라서 비교
        pattern = text[:length]
        # 패턴이 두 번 연속 반복되는지 확인
        if text.startswith(pattern * 2):
            return pattern

In [22]:
# 지역 & 플레이스 기본 정보 문자열 제거를 위한 변수 지정
url2 = 'https://kin.naver.com/qna/list.naver?dirId=12'
headers2 = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response2 = requests.get(url2, headers=headers2)
soup2 = BeautifulSoup(response2.text, 'html.parser')
useless_information = soup2.find(attrs={'class': 'location'}).get_text(strip=True)

def return_location(search):
    a = get_loc(search[:-2]+"01").replace(get_loc(search[:-2]),"")
    a_elim = get_loc(search[:-len(search)]+search[-len(search):-2]).replace(get_loc(search[:-len(search)]),"")
    first_loc = find_pattern(a)
    a = a[len(first_loc):]
    b = get_loc(search).replace(a,"")
    c = b.replace(useless_information, "")
    return c.replace(a_elim,"")

def return_full_location(search):
    full_location = ""
    n = 2
    while n <= len(search):
        full_location += return_location(search[:n])+ " "
        n += 2
    return full_location

# 최종적으로 질문과 답변 정보 받아오는 함수

In [23]:
def collect_information(url, Q_df, A_df):
    # 페이지 요청
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    # BeautifulSoup으로 페이지 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    # 질문 제목 추출
    title = soup.find(attrs={'class': 'endTitleSection'}).get_text(strip=True)
    title = title.replace(",","")
    Q_detail= soup.find(attrs={'class': 'questionDetail'}).get_text(strip=True,separator=' ')
    Q_detail = Q_detail.replace(",","")
    Q_info = soup.find_all(class_='infoItem')
    Q_view = Q_info[0].get_text()[3:]
    Q_date = Q_info[1].get_text()[3:]
    if Q_date.endswith(" 전"):
        Q_date = datetime.today().strftime('%Y.%m.%d')
    if Q_date.startswith("작성일"):
        Q_date = Q_date.replace("작성일", "")
    ans = []
    A_dates = []
    try:
        answers = soup.find_all(class_='se-main-container')
        ans_date = soup.find_all(class_='answerDate')
        for answer in answers:
            ans.append(answer.get_text(strip=True).replace('\u200b', ''))
        for ans_dates in ans_date:
            A_dates.append(ans_dates.get_text(strip=True))
    except:
        answer_error = "아직 답변이 없습니다"
    
    ID = get_dirid(url)
    doc_ID = get_docid(url)
    Q_inf_list =[get_docid(url), title, Q_detail, ID, Q_view, Q_date]
    A_inf_list =[]
    n = 1
    for i,j in zip(ans,A_dates):
        changed = i.replace(","," ")
        if j.endswith(" 전"):
            today = datetime.today().strftime('%Y.%m.%d.')
            A_inf_list.append([doc_ID, doc_ID+str(n), changed[2:],today])
        else:
            A_inf_list.append([doc_ID, doc_ID+str(n), changed[2:],j])
        n += 1
    Q_df.loc[Q_df.shape[0]] = Q_inf_list
    
    for i in A_inf_list:
        A_df.loc[A_df.shape[0]] = i
    return Q_df, A_df

# 질문 리스트에서 각 질문의 URL 값 가져오기

In [24]:
# 질문 목록에서 URL만 가져기기
urls = []
def get_url_from_list(search_url):
    global urls
    response = requests.get(search_url) 
    
    soup = BeautifulSoup(response.text, 'html.parser')

    elements = soup.find_all(class_=re.compile(r'^_nclicks:kls_new\.list'))
    
    for element in elements[1:]:
        href = "https://kin.naver.com"+element.get('href')
        urls.append(href)
    return urls

### list 페이지 설정

In [25]:
for i in range(1,11):
    page = f"https://kin.naver.com/qna/list.naver?dirId=1201&queryTime={datetime.today().strftime('%Y-%m-%d')}%2000%3A25%3A07&page={i}"
    urls = get_url_from_list(page)

In [26]:
urls

['https://kin.naver.com/qna/detail.naver?d1id=12&dirId=12012902&docId=476675172',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=12012106&docId=476735663',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=1201&docId=476733787',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=120110&docId=476731997',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=1201&docId=476731975',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=1201&docId=476731834',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=1201&docId=476731599',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=12012404&docId=476731547',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=12013211&docId=476730994',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=120132&docId=476636453',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=1201&docId=476730686',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=1201&docId=476729776',
 'https://kin.naver.com/qna/detail.naver?d1i

# 실제 데이터 불러오기(현재 약 10분 소요(CPU), 매핑테이블 이용하면 매우 빠르게 끝날 것으로 예상)

In [27]:
for url in tqdm(urls, desc="Processing URLs", ncols=100, leave=True):
    collect_information(url,Q_df, A_df)

Processing URLs: 100%|████████████████████████████████████████████| 200/200 [01:34<00:00,  2.12it/s]


In [28]:
Q_df

,문서 번호,제목,질문,위치 고유번호,조회수,질문 날짜
0,476675172,질문반포 어셔어학원 성심여고 내신대비,성심여고 재학생 중 어셔어학원 다녔다는 후기가 있던데 토플학원이라고 나와있더라고요....,012902,13,2024.10.17
1,476735663,질문신도림 고등학교 가기어렵나요? 신도림고등학교가려는데,신도림 고등학교 가기 어렵나요?,012106,10,2024.10.17
2,476733787,질문서울시티투어버스 노선정보 질문요,노랑버스 빨강버스 2대 있는거같은데 각각 노선이 조금 다른거 같더라구요 주간운행 야...,01,11,2024.10.17
3,476731997,질문홍대 코스 짜주세요,부산에서 남고딩 4명 아이콘 매치 보러 1920일날 올라갑니다. 모텔 홍대쪽에 잡았...,0110,14,2024.10.17
4,476731975,질문서울호서전문학교vs조선이공대,엄마가 대학 전문학교는 안쳐준다면서 취업할때 그래서 조대이공대가서 하라하는데 어디로...,01,11,2024.10.17
...,...,...,...,...,...,...
195,476598052,질문고1 모델 도와주세요,제가 지금 여고를 다니고 있는데 걍 꼴지입니다 공부도 못하고 공부하는 법을 아직도 ...,01,32,2024.10.14
196,476597377,질문고민이네요,저와 제 남자친구는 130일 쯤 됐어요. 오랜만난 사이는 아니에요. 저희는 장거리연...,01,20,2024.10.14
197,476530752,질문양천구 신월동 바자회 어디서 하는건가요?,신월동에서 바자회 한다고 하던데 아시는분 있을까요?,013302,18,2024.10.14
198,476594936,질문종로 야장...,종로 야장 기간이 언제까진가요!?? 여름에만 여나요??아님 몇월달까지 하는지 알려주세여,013757,23,2024.10.14


In [29]:
A_df

,문서 번호,답변 ID,답변,답변 날짜
0,476733787,4767337871,가능하고 1시간정도만 있어도 이동가능합니다.서울 땅덩어리가 그리 크지않습니다.,2024.10.17.
1,476731997,4767319971,추천지 입니다1.장소 입니다방탈출카페 공포.미스터리. 스릴러보드게임카페애견카페...,2024.10.17.
2,476731975,4767319751,이공대 추천합니다,2024.10.17.
3,476731834,4767318341,고구려목장위치: 경기도 양주설명: 고구려목장은 넓은 부지에 젖소와 양떼를 볼 수 ...,2024.10.17.
4,476731834,4767318342,농장도 괜찮은데...멀수도 있어서요~차라리 서울맘카페에 아이랑 동물 볼수 있는곳으...,2024.10.17.
...,...,...,...,...
227,476594936,4765949361,월말까지 합니다,2024.10.14.
228,476594936,4765949362,월 ~ 11월 중반까지는 활발한 영업 시기이고11월 중반이후 부터 비닐쒸우고 밖에...,2024.10.14.
229,476594272,4765942721,친구 가족단위로 갈 수 있는 아주 재미있는 축제를 알아보던 중 대구 이월드 펌킨 ...,2024.10.15.
230,476594272,4765942722,랜드 레츠고,2024.10.14.


In [15]:
Q_df.to_csv('Questions.csv', index = False)
A_df.to_csv('Answers.csv', index = False)